## _*Using Qiskit Aqua for set packing problems*_

Given a collection $S$ of subsets of a set $X$, the set packing problem tries to find the subsets that are pairwise disjoint (in other words, no two of them share an element). The goal is to maximize the number of such subsets.

We will go through three examples to show (1) how to run the optimization in the non-programming way, (2) how to run the optimization in the programming way, (3) how to run the optimization with the VQE.
We will omit the details for the support of CPLEX, which are explained in other notebooks such as maxcut.



### The problem and the brute-force method.

the problem is as follows. First, let us print the list of subsets.

In [1]:
import numpy as np
import json

from qiskit import Aer

from qiskit_aqua import run_algorithm
from qiskit_aqua.input import EnergyInput
from qiskit_aqua.translators.ising import setpacking
from qiskit_aqua.algorithms import ExactEigensolver


input_file = 'sample.setpacking'
with open(input_file) as f:
    list_of_subsets = json.load(f)
    print(list_of_subsets)
    qubitOp, offset = setpacking.get_setpacking_qubitops(list_of_subsets)
    algo_input = EnergyInput(qubitOp)

[[4, 5], [4], [5]]


the brute-force method is as follows. Basically, we exhaustively try all the binary assignments. In each binary assignment, the entry of a subset is either 0 (meaning the subset is not taken) or 1 (meaning the subset is taken). We print the binary assignment that satisfies the definition of the set packing. 

In [2]:
def brute_force():
    # brute-force way: try every possible assignment!
    def bitfield(n, L):
        result = np.binary_repr(n, L)
        return [int(digit) for digit in result]  # [2:] to chop off the "0b" part

    L = len(list_of_subsets)
    max = 2**L
    max_v = -np.inf
    for i in range(max):
        cur = bitfield(i, L)
        cur_v = setpacking.check_disjoint(cur, list_of_subsets)
        if cur_v:
            if np.count_nonzero(cur) > max_v:
                max_v = np.count_nonzero(cur)
    return max_v

size = brute_force()
print("size of set packing", size)

size of set packing 2


### Part I: run the optimization in the non-programming way

In [3]:
params = {
    'problem': {'name': 'ising'},
    'algorithm': {'name': 'ExactEigensolver'}
}
result = run_algorithm(params, algo_input)
x = setpacking.sample_most_likely(len(list_of_subsets), result['eigvecs'][0])
ising_sol = setpacking.get_solution(x)
np.testing.assert_array_equal(ising_sol, [0, 1, 1])
print("size of set packing", np.count_nonzero(ising_sol))

size of set packing 2


### Part II: run the optimization in the programming way

In [4]:
algo = ExactEigensolver(algo_input.qubit_op, k=1, aux_operators=[])
result = algo.run()
x = setpacking.sample_most_likely(len(list_of_subsets), result['eigvecs'][0])
ising_sol = setpacking.get_solution(x)
np.testing.assert_array_equal(ising_sol, [0, 1, 1])
oracle = brute_force()
print("size of set packing", np.count_nonzero(ising_sol))

size of set packing 2


### Part III: run the optimization with VQE

In [5]:
algorithm_cfg = {
    'name': 'VQE',
    'operator_mode': 'paulis'

}

optimizer_cfg = {
    'name': 'SPSA',
    'max_trials': 200
}

var_form_cfg = {
    'name': 'RY',
    'depth': 5,
    'entanglement': 'linear'
}

params = {
    'problem': {'name': 'ising', 'random_seed': 100},
    'algorithm': algorithm_cfg,
    'optimizer': optimizer_cfg,
    'variational_form': var_form_cfg
}
backend = Aer.get_backend('qasm_simulator')
result = run_algorithm(params, algo_input, backend=backend)
x = setpacking.sample_most_likely(len(list_of_subsets), result['eigvecs'][0])
ising_sol = setpacking.get_solution(x)
print("size of set packing", np.count_nonzero(ising_sol))

size of set packing 2
